In [1]:
import numpy as np
import pandas as pd
# import quantstats as qs
import matplotlib.pyplot as plt
import pytz
import os
import cv2
# import pywt

from binance.client import Client
from datetime import datetime, timedelta


## 1. Data Processing (raw -> 성능 좋지 않음)

In [4]:
# Binance API 키 설정
API_KEY = 'nu1AV1q04nvVCGcZpxmYYM4AJovxghU644sTP5Q59Xo4T5tjwsBwLEp4viXQU4om'
API_SECRET = 'yruAAb0iJ1mwz8hsxVIrklph4YWW82Uhb1vlPdiQ1pNMV9EJWOjV1MY0UPHPp7ev'

client = Client(API_KEY, API_SECRET)

# 한국 시간과 UTC와의 시차
# utc_offset = timedelta(hours=9)
utc_offset = timedelta(hours=0)

# 데이터 수집 기간 설정 (3개월)
end_date = datetime.now(pytz.timezone('Asia/Seoul'))  # 현재 한국 시간
start_date = end_date - timedelta(days=10)  # 365일 전까지의 데이터 수집

# start_date = end_date - timedelta(days=90)  # 365일 전까지의 데이터 수집

# UTC 시간을 한국 시간으로 변환하여 데이터 수집
start_date_utc = start_date.astimezone(pytz.utc)
end_date_utc = end_date.astimezone(pytz.utc)


################### Data Load ###################
top_coins_usdt = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "ADAUSDT", "SOLUSDT", "XRPUSDT", "DOTUSDT", "AVAXUSDT", "DOGEUSDT", "LTCUSDT", "WLDUSDT"]

save_root = f"../coin_data/datasets"

if not os.path.exists(save_root):
    os.mkdir(save_root)


for coin in top_coins_usdt:
    # 데이터 수집
    symbol = coin  # 비트코인/달러 페어
    interval = Client.KLINE_INTERVAL_5MINUTE  # 5분 주기 데이터
    candle = client.get_historical_klines(symbol, interval, start_date_utc.strftime("%Y-%m-%d %H:%M:%S"), end_date_utc.strftime("%Y-%m-%d %H:%M:%S"))

    # 데이터를 데이터프레임으로 변환
    columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    df = pd.DataFrame(candle, columns=columns)

    # timestamp 열을 날짜 형식으로 변환 (UTC 시간에서 한국 시간으로 변환)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')

    # 필요한 열만 선택
    df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
    df.reset_index()

    df.columns = ["Date", "Open", "High", "Low","Close", "Volume"]

    df['Open'] = df['Open'].astype(np.float64)
    df['High'] = df['High'].astype(np.float64)
    df['Low'] = df['Low'].astype(np.float64)
    df['Close'] = df['Close'].astype(np.float64)
    df['Volume'] = df['Volume'].astype(np.float64)

    df['Date'] = pd.to_datetime(df['Date'])
    df_ = df.set_index("Date")

    
    ################### Time Series data to Image Data ###################
    df = df_[:].copy()
    length = len(df) // 224
    test_data = df

    mean_data_ = (test_data["Low"].values.astype(np.float64) + test_data["High"].values.astype(np.float64)) / 2
    close_data_ = test_data["Close"].values.astype(np.float64)
    volume_data_ = test_data["Volume"].values.astype(np.float64)

    window_size = 224
        
    print("coin name : ", coin)
    
    zeros = 0
    minus = 0
    plus = 0
    
    for i in range(length):
        mean_data = mean_data_[i* window_size : (i+1) * window_size]
        
        close_data = close_data_[i* window_size : (i+1) * window_size]
        close_copy = close_data.copy()
        
        
        volume_data = volume_data_[i* window_size : (i+1) * window_size]
        lenth_test = len(volume_data)

        if volume_data.all() == 0.:
            volume_data = [1e-04] * lenth_test
            volume_data = np.array(volume_data)

        close_data = np.expand_dims(close_data, axis = 1)
        mean_data = np.expand_dims(mean_data, axis = 1)
        volume_data = np.expand_dims(volume_data, axis = 1)

        total_data = np.concatenate([close_data, mean_data, volume_data], axis = 1)
        
        
        predict_value = close_data_[(i+1) * window_size+1]
        
        interest_rate = (predict_value / close_copy[-1]) -1
        if interest_rate > 0.0005:
            gt = 1
            plus +=1
        elif interest_rate < -0.0005:
            gt = -1
            minus +=1
        else:
            gt = 0
            zeros += 1

        
        save_dir = save_root + f"/{coin}" 
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        save_path = save_dir + "/" +"time_series_" + str(i) +".npy"
        # save_img = save_dir + "/" +"image_" + str(i) +".png"
        save_gt_path = save_dir + "/" +"gt_" + str(i) +".npy"

        np.save(save_path, total_data)
        # # cv2.imwrite(save_img, viz_img)
        np.save(save_gt_path, gt)
        
        
    print("zeros : {}, plus : {}, minus : {}".format(zeros, plus, minus))



coin name :  BTCUSDT
zeros : 1, plus : 3, minus : 8
coin name :  ETHUSDT
zeros : 1, plus : 3, minus : 8
coin name :  BNBUSDT
zeros : 2, plus : 3, minus : 7
coin name :  ADAUSDT
zeros : 2, plus : 3, minus : 7
coin name :  SOLUSDT
zeros : 0, plus : 1, minus : 11
coin name :  XRPUSDT
zeros : 2, plus : 3, minus : 7
coin name :  DOTUSDT
zeros : 3, plus : 3, minus : 6
coin name :  AVAXUSDT
zeros : 1, plus : 4, minus : 7
coin name :  DOGEUSDT
zeros : 2, plus : 4, minus : 6
coin name :  LTCUSDT
zeros : 3, plus : 2, minus : 7
coin name :  WLDUSDT
zeros : 2, plus : 3, minus : 7


## 1-1. Data Processing (normalization and log-scaling)

## 2. Data Saving

In [5]:
from glob import glob
import numpy as np
import os

root_path = f"../coin_data/datasets"

coin_names = os.listdir(root_path)


coin_list = []
gt_list = []


for coin in coin_names :
    coin_paths = sorted(glob(root_path + "/" + coin +"/time_series_*.npy"))
    gt_paths = sorted(glob(root_path + "/" + coin +"/gt_*.npy"))
    
    coin_list.extend(coin_paths)
    gt_list.extend(gt_paths)


In [6]:
import pandas as pd

df =  pd.DataFrame({"original_paths" : coin_list, 
                   "coin_gt_paths" : gt_list})
df.to_csv(f"../coin_data/datasets/train_dataset.csv")

In [7]:
import pandas as pd
import torch
import numpy as np


from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, csv_path, is_train):

        df = pd.read_csv(csv_path)

        img_path = df["original_paths"].tolist()
        gt_path = df["coin_gt_paths"].tolist()

        self.train_data_path, self.test_data_path, self.train_gt_path, self.test_gt_path = train_test_split(img_path, gt_path, random_state=4715, test_size = 0.3)
        self.is_train = is_train
    
    def __len__(self):
        if self.is_train == True:
            return len(self.train_gt_path)
        else:
            return len(self.test_data_path)

    def __getitem__(self, idx):

        if self.is_train:
            data_path = self.train_data_path[idx]
            gt_path = self.train_gt_path[idx]

            original_data = np.load(data_path)
            gt = np.load(gt_path)
            
            
            if gt == -1:
                gt =  np.array([1, 0, 0])
            elif gt == 0:
                gt = np.array([0, 1, 0])
            elif gt == 1:
                gt = np.array([0, 0, 1])


            return original_data, gt
        else:
            data_path = self.train_data_path[idx]
            gt_path = self.train_gt_path[idx]

            original_data = np.load(data_path)
            gt = np.load(gt_path)
            if gt == -1:
                gt =  np.array([1, 0, 0])
            elif gt == 0:
                gt = np.array([0, 1, 0])
            elif gt == 1:
                gt = np.array([0, 0, 1])

            return original_data, gt


## 2. LSTM Defintion

In [8]:
import torch.nn as nn

class CustomLSTM(nn.Module):
    def __init__(self, input_channel =3, output_channel=3, batch_size = 16):
        super(CustomLSTM, self).__init__()
        self.input_channel = input_channel
        self.output_channel = output_channel
        self.batch_size = batch_size
        self.lstm1 = nn.LSTM(input_size=self.input_channel, hidden_size=128, num_layers=3, bidirectional=False, batch_first=True)
        self.activation1 = nn.GELU()
        self.lstm2 = nn.LSTM(input_size=128, hidden_size=256, num_layers=3, bidirectional=False, batch_first=True)
        self.activation2 = nn.GELU()
        self.lstm3 = nn.LSTM(input_size=256, hidden_size=self.output_channel, num_layers=3, bidirectional=False, batch_first=True)
        self.activation3 = nn.GELU()

        self.fc1 = nn.Linear(in_features=self.input_channel * 224, out_features=256)
        self.activation4 = nn.GELU()
        self.fc2 = nn.Linear(in_features= 256, out_features=128)
        self.activation5 = nn.GELU()
        self.fc3 = nn.Linear(in_features=128, out_features=3)
        self.activation6 = nn.Softmax()
        
    def forward(self, x):
        batch, _, _ = x.shape
        x, hidden_tuple1 = self.lstm1(x)
        x = self.activation1(x)
        x, hidden_tuple2 = self.lstm2(x)
        x = self.activation2(x)
        x, hidden_tuple3 = self.lstm3(x)
        x = self.activation3(x)

        x = x.reshape(batch, -1)
        x = self.fc1(x)
        x = self.activation4(x)
        x = self.fc2(x)
        x = self.activation5(x)
        x = self.fc3(x)
        x = self.activation6(x)
        return x
        
model = CustomLSTM(input_channel=3, output_channel=3)

In [14]:
model

CustomLSTM(
  (lstm1): LSTM(3, 128, num_layers=3, batch_first=True)
  (activation1): GELU(approximate='none')
  (lstm2): LSTM(128, 256, num_layers=3, batch_first=True)
  (activation2): GELU(approximate='none')
  (lstm3): LSTM(256, 3, num_layers=3, batch_first=True)
  (activation3): GELU(approximate='none')
  (fc1): Linear(in_features=672, out_features=256, bias=True)
  (activation4): GELU(approximate='none')
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (activation5): GELU(approximate='none')
  (fc3): Linear(in_features=128, out_features=3, bias=True)
  (activation6): Softmax(dim=None)
)

## Train Loop

In [18]:
import torch.optim as optim
import wandb 

from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold

train_dataset = CustomDataset("../coin_data/datasets/train_dataset.csv", True)
val_dataset = CustomDataset("../coin_data/datasets/train_dataset.csv", False)

epoches = 100
device = "cuda" if torch.cuda.is_available() else "cpu" 

### 1. model 정의
model = CustomLSTM()
model = model.to(device)
## 2. optimizer 정의
optimizer = optim.AdamW(model.parameters(), lr = 1e-04, betas = (0.9, 0.999))
## 3. Scheduler 정의
scheduler = CosineAnnealingLR(optimizer, T_max=10)
## 4. criterion 정의
criterion = nn.CrossEntropyLoss()

train_dataset = CustomDataset("../coin_data/datasets/train_dataset.csv", True)
test_dataset = CustomDataset("../coin_data/datasets/train_dataset.csv", False) 
test_loader = DataLoader(test_dataset, batch_size = 16, shuffle = False)

kfold = KFold(n_splits=5, shuffle=True, random_state=4715)

# watch the train step
wandb.init()

for epoch in range(epoches):
    train_losses = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(train_dataset)):
        train_subset = torch.utils.data.Subset(train_dataset, train_idx)
        train_subset_loader = DataLoader(train_subset, batch_size = 16, shuffle = True)
        val_subset = torch.utils.data.Subset(train_dataset, val_idx)
        val_subset_loader = DataLoader(val_subset, batch_size = 16, shuffle = False)


        train_loss_list = []
        for (t_idx), (train_data, train_gt) in enumerate(train_subset_loader):
            train_data = train_data.to(device)
            train_data = train_data.to(torch.float)
            train_gt = train_gt.to(device)
            train_gt = train_gt.to(torch.float)


            train_prediction = model(train_data)

            train_loss = criterion(train_prediction, train_gt)
            
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            train_loss_list.append(train_loss.item())

            break
        val_loss_list = []
        with torch.no_grad():
            for (v_idx), (val_data, val_gt) in enumerate(val_subset_loader):
                val_data = val_data.to(device)
                val_data = val_data.to(torch.float)
                val_gt = val_gt.to(device)
                val_gt = val_gt.to(torch.float)

                val_prediction = model(val_data)

                val_loss = criterion(val_prediction, val_gt)
                val_loss_list.append(val_loss.item())

    test_loss_list = []
    with torch.no_grad():
        for test_idx, (test_data, test_gt) in enumerate(test_loader):
            test_data = test_data.to(device)
            test_data = test_data.to(torch.float)
            test_gt = test_gt.to(device)
            test_gt = test_gt.to(torch.float)

            test_prediction = model(test_data)

            test_loss = criterion(test_prediction, test_gt)
            test_loss_list.append(test_loss.item())

        
        break
    break

c:\Users\JooEun Moon\anaconda3\envs\trading_py64\lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [ ]:

run = wandb.init(
    # Set the project where this run will be logged
    project="my-awesome-project",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": 0.01,
        "epochs": 10,
    },
)

## 4. Model Architecture

In [29]:
import torch.nn as nn
import torch.nn.functional as F

class ModifiedDinoVisionTransformer(nn.Module):
    def __init__(self):
        super(ModifiedDinoVisionTransformer, self).__init__()
        
        # 기존 DinoVisionTransformer 모델 로드
        self.dino_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
        
        # 추가 레이어 정의
        self.additional_layer = nn.Linear(384, 3)  # 입력 크기: 384, 출력 크기: 3

    def forward(self, x):
        x = self.dino_model(x)
        x = self.additional_layer(x)
        x = F.softmax(x, dim=-1)  # Softmax 레이어 추가
        return x


## 5. Train loop

In [30]:
import torch.optim as optim
import os
import wandb

from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR

train_dataset = CustomDataset("/home/wondong/coin/DinoV2/dataset_scelo/train_dataset.csv", True)
test_dataset = CustomDataset("/home/wondong/coin/DinoV2/dataset_scelo/train_dataset.csv", False)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size = 32)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size = 32)

model = ModifiedDinoVisionTransformer()

epoch_size = 3000
device =  "cuda" if torch.cuda.is_available() == True else "cpu"
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# AdamW 옵티마이저 정의
optimizer = optim.AdamW(model.parameters(), lr=0.000001)

# 코사인 스케줄러 정의
scheduler = CosineAnnealingLR(optimizer, T_max=10)

save_path = "/home/wondong/coin/DinoV2/checkpoints2"
if not os.path.exists(save_path):
    os.mkdir(save_path)
save_checkpoints = save_path + "/best_aug.pt"
min_loss = 100


wandb.init(project="crypo", name="scalogram_2")

for epoch in tqdm(range(epoch_size)):
    train_losses = []
    for train_idx, (train_img, train_gt) in enumerate(train_dataloader):

        train_img = train_img.to(device)
        train_img = train_img.to(dtype = torch.float)
        
        train_gt = train_gt.to(device)
        train_gt = train_gt.to(dtype = torch.float)

        optimizer.zero_grad()

        train_pred = model(train_img)

        train_loss = criterion(train_pred, train_gt)
        train_loss = train_loss
        train_losses.append(train_loss.item())
        train_loss.backward()
        optimizer.step()
    train_losses_np = np.array(train_losses)
    mean_train_loss = np.mean(train_losses_np)

    print("Mean of Train Loss : ", mean_train_loss)

    val_losses = []
    with torch.no_grad():
        for val_idx, (val_img, val_gt) in enumerate(test_dataloader):

            val_img = val_img.to(device)
            val_img = val_img.to(dtype = torch.float)
        
            val_gt = val_gt.to(device)
            val_gt = val_gt.to(dtype = torch.float)

            val_pred = model(val_img)

            val_loss = criterion(val_pred, val_gt)
            val_loss = val_loss 
            val_losses.append(val_loss.item())
  

    val_losses_np = np.array(val_losses)
    mean_val_loss = np.mean(val_losses_np)


    print("Mean of Val Loss : ", mean_val_loss)
    
    scheduler.step()

    wandb.log({"train loss": mean_train_loss, "validation loss": mean_val_loss})

    if min_loss > mean_val_loss:
        min_loss = mean_val_loss
        print("")
        print("#"*50)
        print("Wow New Model Pop up : ", min_loss)
        print("#"*50)
        

        if not os.path.exists(save_path):
            os.mkdir(save_path)
        
        torch.save(model.state_dict(), save_checkpoints)


Using cache found in /home/wondong/.cache/torch/hub/facebookresearch_dinov2_main
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_pretrain.pth" to /home/wondong/.cache/torch/hub/checkpoints/dinov2_vits14_pretrain.pth
100%|██████████| 84.2M/84.2M [00:12<00:00, 7.20MB/s]


train loss,████▇▆▆▆▆▅▄▄▄▄▃▄▃▃▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▂▁▁▂▁▁
validation loss,▁▁▁▂▂▂▃▂▃▃▃▃▄▄▅▃▅▆▅▅▅▆▆▆▆▆▆▆▆▇▆▆▆▇█▇▆▆▇▇
train loss,0.60625
validation loss,1.13369


  0%|          | 0/3000 [00:00<?, ?it/s]

Mean of Train Loss :  1.0463710927088326


  0%|          | 1/3000 [03:44<186:52:18, 224.32s/it]

...

  1%|          | 22/3000 [1:22:13<187:02:45, 226.11s/it]

Mean of Val Loss :  1.0839455153188136


  1%|          | 22/3000 [1:22:20<185:44:58, 224.55s/it]


KeyboardInterrupt: 

wandb: Network error (ConnectionError), entering retry loop.
